## Import library

In [6]:
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

## Import dataset

In [7]:
df = pd.read_csv("data/kompas.csv")
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


## Extract TFIDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matriks = tfidf.fit_transform(df['teks'])

## TFIDF Similarity digunakan untuk mendapatkan document similarity

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
# untuk membandingkan antara value di index pertama dengan seluruh index di tfidf_matriks dari index 1 sampai n.
# sistem kerja cosine adalah mencari terlebih dahulu dua sudut lalu dicosinus kan
sim = cosine_similarity(tfidf_matriks[0], tfidf_matriks)
sim

array([[1.        , 0.00858328, 0.01060043, ..., 0.00856287, 0.00677808,
        0.01513341]])

In [17]:
# untuk melihat di index berapa yang memiliki value tinggi. yang paling mirip dari kanan yaitu 0, 144, 215, dst
sim.argsort()

array([[ 932, 1131, 1593, ...,  215,  144,    0]], dtype=int64)

In [30]:
# index lalu jumlah kalimatnya
df.teks[0][:200]

'Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri\nKejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan'

In [31]:
# index lalu jumlah kalimatnya
df.teks[144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [32]:
df.teks[215][:200]    

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pag'

In [33]:
df.teks[932][:200]    

'Pusat Pengolahan Ikan Asin Lhokseumawe Terbakar\nPusat pengolahan ikan asin di kawasan Pasar Ikan Lhokseumawe (Aceh) terbakar bersama berton-ton stok ikan asin Sabtu (13/1) siang. Sedikitnya 25 gudang '

## Keyword Extraction 

In [37]:
vocab = tfidf.get_feature_names()
vocab[-10:]

['zuniga memilih',
 'zunnatul',
 'zunnatul mafruhah',
 'zurich',
 'zurich northholt',
 'zw',
 'zw suparman',
 'zw tim',
 'zx',
 'zx diserbu']

In [38]:
tfidf_matriks[0].toarray()

array([[0.02115058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [39]:
sorted_tfidf = tfidf_matriks[0].toarray()[0].argsort()
sorted_tfidf

array([274201, 365453, 365452, ..., 386361, 436633, 169209], dtype=int64)

In [43]:
vocab[169209]

'ginandjar'

In [44]:
vocab[274201]

'litbang partai'

In [49]:
# melihat 10 vocab atau keyword unik di tiap features teratas
[vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

In [56]:
def extract_keywords_tfidf(doc, tfidf, topk=10) :
    matrix = tfidf.transform([doc])
    vocab = tfidf.get_feature_names()
    
    sorted_tfidf = matrix[0].toarray()[0].argsort()
    return [vocab[idx] for idx in reversed(sorted_tfidf[-topk:])]

In [57]:
teks = """ 
Unggahan IG Persija Isyaratkan Bambang Pamungkas Gantung Sepatu?

Unggahan akun Instagram Persija pada Minggu, 15 Desember 2019 lalu, sempat membuat geger dunia maya, terutama twitter. Menjelang perhelatan melawan Persebaya Surabaya di pecan ke-33 Liga 1 2019 yang akan digelar Selasa, 17 Desember nanti, IG Persija memposting foto dan caption yang tak biasa.

Unggahan tersebut mengusung foto pemain seior Bambang Pamungkas (Bepe) dengan tulisan Almost There pada gambar. Sementara di bagian caption, tertulis ‘Laga Home Pamungkas’. Berkat unggahan ini, banyak pihak berspekulasi bahwa Bepe akan segera gantung sepatu alias pensiun.

Hal ini diperkuat dengan postingan Bepe dengan ungguhan yang sama beserta tulisan Almost There. Terlebih, isu pensiun ini telah berhembus sejak beberapa waktu lalu. CEO Persija, Ferry Paulus sampai kini belum bisa memberikan konfirmasi pasti akan kebenaran berita ini.

Jika ditilik dari umurnya yang sudah menginjak 38 tahun, Bepe memang sudah sewajarnya mengambil jatah pensiun. Namun, keinginan tersebut tahun lalu sempat dicekal bos Persija Gede Widiade lantaran ia dianggap sebagai ikon yang menjanjikan.

"""

In [58]:
extract_keywords_tfidf(teks, tfidf)

['persija',
 'pamungkas',
 'pensiun',
 'almost',
 'there',
 'bambang pamungkas',
 'gantung',
 'foto',
 'tulisan',
 'sepatu']

In [59]:
tfidf.idf_.argsort()

array([ 28062,      0, 200981, ..., 199875, 199906, 548403], dtype=int64)

In [63]:
# paling sering muncul
vocab[548403]

'zx diserbu'

In [64]:
# paling jarang muncul
vocab[28062]

'``'